# Homework09

Exercises to practice dimensionality reduction with PCA

## Goals

- Add unsupervised pre-processing to our ML flow: encode -> normalize -> pre-process -> train -> evaluate
- Build an intuition for PCA and when/how to use it
- Understand the benefits of dimensionality reduction

### Setup

Run the following 2 cells to import all necessary libraries and helpers for this homework.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/image_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/Homework07/raw/main/Homework07_utils.pyc

!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/releases/latest/download/0801-500.tar.gz | tar xz

In [ ]:
import PIL.Image as PImage

from os import listdir, path

from data_utils import PCA, RandomForestClassifier

from image_utils import get_pixels, make_image

from Homework07_utils import CamUtils

## PCA and Classification

Let's practice doing some more image classification with `PCA` and analyzing the results.

The dataset is the same one we used in `Homework07`. It has images from $25$ different security cameras, and our task is to separate them by camera. Some of the cameras move, some of them don't, and there are more than $1000$ images, so there's no way we want to do this by hand.

Let's start by loading the training images into a list of pixels, like we did with the `faces` dataset in class.

### Loading Data

If we look at the images in `./data/image/0801-500/train/`, we'll notice that they are named and organized in a slightly different way. They're all in the same directory and the first part of their filename specifies which camera they came from. Even though those `ids` are numbers, they're not sequential, so we'll use some helper functions to extract a unique `label` from their filenames.

This is exactly what the `OrdinalEncoder` class does, but since we only have to encode this one column, we'll do it by hand while we read the files in.

In [ ]:
# this creates a list of all the files in a given directory, that end in .jpg
train_files = [f for f in listdir("./data/image/0801-500/train") if f.endswith(".jpg")]

In [ ]:
# TODO: check and see what is inside the list here
print(len(train_files))
print(train_files[0])
print(train_files[100])
print(train_files[-1])

Now we'll read the image pixels and extract their labels. `CamUtils.get_label()` is the helper function we'll use to "encode" and return a label id based on the filename.

In [ ]:
pixel_data = []
label_data = []

for fname in train_files:
  label = CamUtils.get_label(fname)
  img = PImage.open(path.join("./data/image/0801-500/train", fname))
  pixel_data.append(get_pixels(img))
  label_data.append(label)

In [ ]:
# TODO: check if labels got extracted correctly by looking at 
#       the first few items of the label list and the filename list

print(label_data[0])
print(train_files[0])

print(label_data[1])
print(train_files[1])

print(label_data[10])
print(train_files[10])


Like in `Homework07`, the labels and the filenames won't match exactly since labels start at $0$ and the filenames start at $01$ and skip some numbers.

We can open some images from pixels, just to make sure we loaded them correctly:

In [ ]:
display(make_image(pixel_data[0], width=256))
display(make_image(pixel_data[10], width=256))

### PCA

Create a `DataFrame` called `cam_pca_df` by running `PCA` on the list of pixel data, just like we did in class, and then append a `label` column to it, just like we did in Homewokr07 with: `train_df["label"] = label_data`.

You can change how many components to extract from the images later, but for this first run just use $10$ components so we can compare classification with $10$ `PCA` components to the classification with $10$ random features from `Homework07`.  Make sure to confirm how much of the `variance` is preserved by this transformation.

In [ ]:
# TODO: run PCA, get DataFrame
pixel_pca = PCA(n_components=10)
cam_pca_df = pixel_pca.fit_transform(pixel_data)

# TODO: add "label" column
cam_pca_df["label"] = label_data

In [ ]:
# TODO: print variance
print(pixel_pca.explained_variance())

# TODO: take a look at the first 5 rows
cam_pca_df.head()

### Interpretation

<span style="color:hotpink">
How many components did you keep ?<br>
Did you try different numbers ?<br>
What percent of variance is kept by this transformation?
</span>

10 PCA components results in ~0.78 variance.

5 PCA components --> ~0.65 variance

1 PCA component --> ~0.24 variance

1 versus 10 PCA components is a very large difference, but 5 to 10 is less of a difference so depending on computation speed and ideal minimum variance it could be worth finding a sweet spot somewhere between 5 and 10. 

### Reconstruct

Before we go run our classification, let's take a look at some reconstructed images, just to make sure they make sense.

Take a look at reconstructed versions of images $0$, $10$, and whichever other images you might have opened above.

We did this in class. It involves a `inverse_transform()` function.

In [ ]:
pca_pixels = pixel_pca.inverse_transform(cam_pca_df)

# TODO: reconstruct image at index 0
display(make_image(pixel_data[0], width=256))
display(make_image(pca_pixels.loc[0], width=256))

# TODO: reconstruct image at index 10
display(make_image(pixel_data[10], width=256))
display(make_image(pca_pixels.loc[10], width=256))


# the reconstruction of image 0 looks surprisingly bad... but image 10 looks okay


The reconstructed images should look blurrier and noisier than the original ones, but they should still look something like the original ones.

### Classify !

Train a `RandomForestClassifier` on the `PCA` data.

Remember to separate the output feature (`label`) from the input features (`PC0`, `PC1`, etc).

In [ ]:
# TODO: create a classifier
model_rf = RandomForestClassifier()

# TODO: split input and output features
chosen_columns = cam_pca_df.columns[:10]
train_features = cam_pca_df[chosen_columns]

out_features = cam_pca_df["label"]

# TODO: fit the model
model_rf.fit(train_features, out_features)

# TODO: run predictions
train_predictions = model_rf.predict(train_features)

### Measure Error

For the train set we could use the generic `classification_error()` function that we've used before because we have the true labels for each file, but for measuring accuracy for our `test` set classification, which has secret labels, we have to use a more specific function from the `CamUtils` class.

The function is called `classification_accuracy()` and it takes $2$ parameters, a list of filenames and the predicted labels associated with those filenames.

The result is an accuracy score ratio, between $0.0$ and $1.0$.

We can run this function on the `train` set as well:

In [ ]:
# train_files is the list of files we got way up above
CamUtils.classification_accuracy(train_files, train_predictions)

### Interpretation

<span style="color:hotpink">
What does this mean?<br>
Should we expect the same result from the test dataset?<br>
Why, or why not?
</span>

Since the training data has labels they can be classified accurately 100% of the time as the model was trained on that data. The test data does not have labels and will be classified based on the model so the accuracy will be less, but will be a more valid measurement of how good our model is. 

### Run on Test set

Now we'll open the image files inside the `./data/image/0801-500/test/` directory, using almost the exact same steps as we did above to create a `DataFrame` with the exception that we don't have labels for these images.

In [ ]:
# TODO: create a list of files in the test/ directory
test_files = [f for f in listdir("./data/image/0801-500/test") if f.endswith(".jpg")]

# TODO: check its length and content
display(test_files[0:10])
print(len(test_files))

We'll use a similar loop to extract the pixel information from these images, except we don't have `label` data, just pixels.

In [ ]:
test_pixel_data = []

for fname in test_files:
  img = PImage.open(path.join("./data/image/0801-500/test", fname))
  test_pixel_data.append(get_pixels(img))

Now we can run `PCA` and the `RandomForestClassifier`.

Since those are already trained, all we have to do is run their `transform()` function on the data.

In [ ]:
# TODO: run PCA on test pixels, get DataFrame
#       this is the same PCA that was created above, not a new one

cam_pca_test_df = pixel_pca.fit_transform(test_pixel_data)

In [ ]:
# TODO: take a look at the first 5 rows
cam_pca_test_df.head()

It's always a good idea to check the result of the `PCA` transformation by looking at the `DataFrame` and reconstructing some images.

In [ ]:
# TODO: reconstruct one or two images

pca_pixels_test = pixel_pca.inverse_transform(cam_pca_test_df)

# reconstruct image at index 0
display(make_image(test_pixel_data[0]))
display(make_image(pca_pixels_test.loc[0]))

# reconstruct image at index 10
display(make_image(test_pixel_data[10]))
display(make_image(pca_pixels_test.loc[10]))

### Interpretation

<span style="color:hotpink"><br>
The reconstructed images from the `test` set will always be noisier than the `train` images.<br><br>
Why?<br><br>
</span>

Because the model was fit on the training data, not the test data, and we only retained ~0.78 of the original data from the PCA transformation.



Run the classifier and get predicted labels. Then get its accuracy ratio is by running the `CamUtils.classification_accuracy()` function with the list of filenames and list of predictions.

In [ ]:
# also separate test dataset features
test_features = cam_pca_test_df[chosen_columns]

# TODO: run classifier. This is the same classifier already fitted on the train data above
test_predictions = model_rf.predict(test_features)

# TODO: get accuracy
CamUtils.classification_accuracy(test_files, test_predictions)

### Interpretation

<span style="color:hotpink">
What's the accuracy ? How could we improve that if we wanted ?<br>
How does this compare to the classification model from <code>Homework07</code>? They can both be tuned to perform well, but how many features did we need last time in order to get an accuracy comparable to this?
</span>

The accuracy for the test data is ~0.35, which seems very low and seems like I probably did something wrong. The accuracy from HW07 from just using the first 10 columns was ~0.64, which is significantly higher than using 10 PCA components, and I would expect to have to use fewer PCA columns to get a comparable accuracy. 